In [2]:
from lxml import etree
import numpy as np

In [12]:
# TODO

skyweave_mass = 20.1  # kg

num_elements_x = 2
length_x = 0.4  # m
num_elements_y = 2
length_y = 0.4  # m

num_elements = num_elements_x * num_elements_y
element_mass = skyweave_mass / num_elements

sdf = etree.Element("sdf", version="1.7")
robot = etree.SubElement(sdf, "model", name="spring_damper_mesh")
# pose = etree.SubElement(robot, "pose")
# pose.text = "0 0 0 0 0 0"
static = etree.SubElement(robot, "static")
static.text = "false"

# create a grid of point masses
pt_masses = []
for idx_y in range(num_elements_y):
    pt_masses.append([])
    for idx_x in range(num_elements_x):
        m = etree.SubElement(
            robot,
            "link",
            name=f"mass_x{idx_x}_y{idx_y}",
        )
        pose = etree.SubElement(m, "pose")
        pose.text = f"{(idx_x)*(length_x/(num_elements_x-1))} {(idx_y)*(length_y/(num_elements_y-1))} 0 0 0 0"
 
        inertial = etree.SubElement(m, "inertial")
        mass = etree.SubElement(inertial, "mass")
        mass.text = str(element_mass)
        inertia = etree.SubElement(inertial, "inertia")

        # define all inertia components and their values
        inertia_values = {
            "ixx": "1e-5", "ixy": "0", "ixz": "0",
            "iyy": "1e-5", "iyz": "0",
            "izz": "1e-5"
        }

        # create them in a loop
        for name, value in inertia_values.items():
            etree.SubElement(inertia, name).text = value

        collision = etree.SubElement(m, "collision", name=f"collision_x{idx_x}_y{idx_y}")
        geometry = etree.SubElement(collision, "geometry")
        sphere = etree.SubElement(geometry, "sphere")
        radius = etree.SubElement(sphere, "radius")
        radius.text = "0.03"

        visual = etree.SubElement(m, "visual", name=f"visual_x{idx_x}_y{idx_y}")
        geometry_v = etree.SubElement(visual, "geometry")
        sphere_v = etree.SubElement(geometry_v, "sphere")
        radius_v = etree.SubElement(sphere_v, "radius")
        radius_v.text = "0.03"


        pt_masses[idx_y].append(m)
        # create 4 frames around each point mass to add universal joints
        # Turns out frames can't be parents of joints, so like, at the moment, I don't see the point of it
        # frame_N = etree.SubElement(m, "frame", name=f"m_x{idx_x}_y{idx_y}_N", attached_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_N = etree.SubElement(frame_N, "pose", relative_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_N.text = f"{0.5*length_x/num_elements_x} 0 0 0 0 0"
        # frame_W = etree.SubElement(m, "frame", name=f"m_x{idx_x}_y{idx_y}_W", attached_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_W = etree.SubElement(frame_W, "pose", relative_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_W.text = f"0 0 0 0 0 {np.pi/2}"
        # frame_S = etree.SubElement(m, "frame", name=f"m_x{idx_x}_y{idx_y}_S", attached_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_S = etree.SubElement(frame_S, "pose", relative_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_S.text = f"0 0 0 0 0 {np.pi}"
        # frame_E = etree.SubElement(m, "frame", name=f"m_x{idx_x}_y{idx_y}_E", attached_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_E = etree.SubElement(frame_E, "pose", relative_to=f"mass_x{idx_x}_y{idx_y}")
        # pose_E.text = f"0 0 0 0 0 {-np.pi/2}"

joint_test = etree.SubElement(
    robot,
    "joint",
    name="test_joint",
    type="revolute",
)
parent = etree.SubElement(joint_test, "parent")
parent.text = "mass_x0_y0"
child = etree.SubElement(joint_test, "child")
child.text = "mass_x1_y0"
axis = etree.SubElement(joint_test, "axis")
xyz = etree.SubElement(axis, "xyz")
xyz.text = "0 1 0"
pose = etree.SubElement(joint_test, "pose")
pose.text = f"-0.2 0 0 0 0 0"


# link adjacent point masses with yz universal joints
for idx_y in range(num_elements_y):
    for idx_x in range(num_elements_x):
        # check if north connection is possible, if so create joint
        pass
        # check if west connection is possible, if so, create joint
    
# link diagonal point masses with shear springs attached to ball joints to prevent shear


with open("spring_damper_mesh.sdf", "wb") as f:
    f.write(etree.tostring(sdf, pretty_print=True))